In [6]:
import numpy as np 
import pandas as pd
import json 
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [2]:
!wget -q -O 'toronto_data.json' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


In [15]:
with open('toronto_data.json') as origin:
    soup = BeautifulSoup(origin, 'html.parser')


In [62]:
match = soup.find_all('td') #class_="wikitable sortable"    # get all the attributes
match

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td>, <td>M4A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
 </td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
 </td>, <td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights

In [82]:
from operator import itemgetter 
content,Postcode,Borough,Neighborhood =[],[],[],[]
for item in match:                              
    if item.text == "":
        break
    else:
        content.append(item.text)
   #print(content)
   # print(Postcode,Borough,Neighborhood)
index1  = np.arange(0,len(content),3)
index2  = np.arange(1,len(content),3)
index3  = np.arange(2,len(content),3)
Postcode = itemgetter(*index1)(content)                      # assign values to each column
Borough = itemgetter(*index2)(content)
Neighborhood =itemgetter(*index3)(content)
Neighborhood = [x.rstrip() for x in itemgetter(*index3)(content)]
#print(len(Postcode),len(Borough),len(Neighborhood))

In [105]:
#pd.set_option('display.max_rows', None)
toronto = pd.DataFrame(data = {'Postcode':Postcode,'Borough':Borough,'Neighborhood':Neighborhood})  # create the dataframe
toronto = toronto[toronto['Borough']!='Not assigned'].reset_index(drop = True)   

for i,loc in enumerate(toronto['Neighborhood']):                                           #replace the not assigned
    if loc =='Not assigned':
        toronto.at[i,"Neighborhood"] = toronto.at[i,"Borough"] 
"Not assigned" in toronto.Neighborhood.values     # check if cleared 

df = toronto.groupby(['Postcode','Borough'])['Neighborhood'].apply(",".join).reset_index()  #combine the 'Neighborhood'

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [106]:
df.shape

(103, 3)